In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
df_train = pd.read_csv('dataset/training.csv')
df_test = pd.read_csv('dataset/testing.csv')
df_test_order = pd.read_csv('test.csv')

In [3]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [4]:
df_train.head()

,bidder_id,time_delta_auction_bids_mean,time_delta_auction_bids_std,time_delta_auction_bids_min,time_delta_auction_bids_25%,time_delta_auction_bids_50%,time_delta_auction_bids_75%,time_delta_auction_bids_max,time_delta_bids_in_auction_median,time_delta_bids_in_auction_kurtosis,time_delta_user_bids_mean,time_delta_user_bids_std,time_delta_user_bids_min,time_delta_user_bids_25%,time_delta_user_bids_50%,time_delta_user_bids_75%,time_delta_user_bids_max,time_delta_user_bids_median,time_delta_user_bids_kurtosis,bids_with_bot_ip,total_bids,ip_shared_with_bots,unique_ip,avg_bids_per_url,device_bids_count,device_bids_mean,device_bids_std,device_bids_min,device_bids_25%,device_bids_50%,device_bids_75%,device_bids_max,time_delta_auction_user_bids_mean,time_delta_auction_user_bids_std,time_delta_auction_user_bids_min,time_delta_auction_user_bids_25%,time_delta_auction_user_bids_50%,time_delta_auction_user_bids_75%,time_delta_auction_user_bids_max,first_bid_over_total_bids,win_bid_over_total_bids,outcome
0,668d393e858e8126275433046bbd35c6tywop,3.455380,4.189127,0.000000,0.842105,2.210526,4.526316,70.315789,2.210526,28.351440,0.682536,0.833721,0.0,0.157895,0.421053,0.894737,10.315789,0.421053,16.296545,9752,19988,1738,10316,4.415286,1029.0,19.424684,126.555472,1.0,1.0,3.0,9.0,3753.0,19.424684,126.555472,1.0,1.0,3.0,9.0,3753.0,0.009306,0.009356,0.0
1,e8291466de91b0eb4e1515143c7f74dexy2yr,6.064759,8.628476,0.000000,1.421053,3.526316,7.526316,137.315789,3.526316,52.546345,0.453802,0.565775,0.0,0.105263,0.263158,0.578947,14.052632,0.263158,26.490827,7597,30062,1916,20492,101.905085,403.0,74.595533,298.915257,1.0,1.0,3.0,12.5,4047.0,74.595533,298.915257,1.0,1.0,3.0,12.5,4047.0,0.004324,0.004690,0.0
2,a58ace8b671a7531c88814bc86b2a34cf0crb,44.098620,82.762948,0.052632,4.684211,14.000000,45.263158,832.368421,14.000000,32.037707,0.070843,0.102856,0.0,0.000000,0.052632,0.105263,3.157895,0.052632,48.097035,123814,192565,2639,53405,5.273153,2020.0,95.329208,726.211904,1.0,2.0,5.5,27.0,26560.0,95.329208,726.211904,1.0,2.0,5.5,27.0,26560.0,0.001916,0.002088,0.0
3,7e34a26c4c339e875997f4b9239d2b5fscng4,6.856177,9.265117,0.000000,1.894737,4.157895,8.526316,104.052632,4.157895,33.984406,1.497714,2.715410,0.0,0.315789,0.736842,1.736842,77.210526,0.736842,174.020783,1834,9108,363,6046,4.829268,367.0,24.817439,125.091734,1.0,1.0,3.0,11.0,2176.0,24.817439,125.091734,1.0,1.0,3.0,11.0,2176.0,0.000878,0.000988,0.0
4,6c8ffec692e88d81e1b18d47818abb04v6llr,234.925590,353.486612,1.000000,38.263158,121.947368,285.934211,1796.947368,121.947368,9.267263,0.161788,0.203371,0.0,0.052632,0.105263,0.210526,3.052632,0.105263,16.690454,53392,84319,2572,27560,6.284958,1610.0,52.372050,487.799808,1.0,1.0,4.0,16.0,18438.0,52.372050,487.799808,1.0,1.0,4.0,16.0,18438.0,0.004115,0.004186,0.0


## Random Forest Example

In [5]:
# Set random seed
np.random.seed(0)

In [6]:
# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = RandomForestClassifier(n_jobs=1000, random_state=0)

X_train = df_train.drop(['outcome', 'bidder_id'], axis=1)
y_train = df_train['outcome']


clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1000, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [7]:
X_train.shape, y_train.shape

((1984, 40), (1984,))

## On Training

In [8]:
y_predict = clf.predict(X_train)

accuracy_score(y_train, y_predict)

0.99647177419354838

## On Testing

In [9]:
X_test = df_test.drop(['bidder_id'], axis=1)

y_predict = clf.predict(X_test)

X_id = df_test['bidder_id']

In [10]:
result_map = {}

for i in range(len(y_predict)):

    result_map[X_id[i]] = y_predict[i]

In [11]:
df_test_order['prediction'] = 0
df_test_order = df_test_order[['bidder_id', 'prediction']]

## Submition requires 4700 entries, have to add entries with missing data

In [12]:
for bidder_id in result_map:
    df_test_order.loc[df_test_order['bidder_id'] == bidder_id, 'prediction'] = result_map[bidder_id]

In [13]:
df_test_order.shape

(4700, 2)

In [14]:
df_test_order.to_csv('predictions/submission_1.csv', index=None)